In [ ]:
#Librerías y paquetes

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams["figure.figsize"]=(16,9)
plt.style.use("fast")

from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Cargar datos

df = pd.read_csv(r"C:\Users\carlo\OneDrive\Escritorio\JAVERIANA\tserie\tserie.csv", delimiter=';', decimal=',')  
df.info()
print(df)

In [ ]:
df['unds'] = df['unds'].astype('float64')
df

In [ ]:
# Estadística descriptiva de la base

df.describe


In [ ]:
# Separar y

y = df['unds']
print(y)

In [ ]:
# Graficar serie

plt.plot(y)


In [ ]:
# escalar datos (-1,1)

scaler=MinMaxScaler(feature_range=(-1,1))

y=y.reshape(-1,1)

y=scaler.fit_transform(y)

y
        
                          

In [ ]:
# Convertir los datos para un problema supervisado
#https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

PASOS=7

from pandas import DataFrame
from pandas import concat


def series_to_supervised(y, n_in=1, n_out=1, dropnan=True):

	n_vars = 1 if type(y) is list else y.shape[1]
	df = pd.DataFrame(y)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


datos=series_to_supervised(y,PASOS)
datos


In [ ]:
#Separar en datos de entrenamiento y prueba (OJO manteniendo el orden temporal)

values=datos.values
n_train_days=604-(30+PASOS)
train=values[:n_train_days, :]
test=values[n_train_days:, :]


#Separar entradas y salidas

x_train, y_train=train[:, :-1],train[:, :-1]
x_test, y_test=test[:, :-1],test[:, :-1]


x_train=x_train.reshape((x_train.shape[0],1,x_train.shape[1]))
x_test=x_test.reshape((x_test.shape[0],1,x_test.shape[1]))
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)


In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(1, 7)))
model.add(Dense(7, activation='tanh'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='mean_absolute_error',metrics=["mse"],optimizer='Adam')

In [ ]:
# Optimizar el modelo

model.fit(x_train,y_train,epochs=1000)
print(model.summary())


In [ ]:
prediccion=model.predict(x_test)
prediccion


In [ ]:
x_new=[[0.38,0.02,1,-0.34,-0.45,-0.45,0.1]]
y_new=model.predict(x_new)

print(y_new)